In [1]:
import numpy as np
import data
from timeit import default_timer as timer

In [2]:
train = data.train
test = data.test

In [13]:
class MatrixFactorize():
    def __init__(self, train, test, k, learning_rate, reg_param, epochs, verbose = False):
        self.R = train
        self.test = test
        self.l = np.array(np.vectorize(lambda x: 0 if x == 0 else 1)(train), dtype = np.float64)
        self.mask = self.l == 1
        self.N = 1./np.linalg.norm(self.l, axis = 1)
        self.num_users, self.num_items = test.shape
        self.k = k
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.epochs = epochs
        self.verbose = verbose
        
    def fit(self):
        self.P = np.random.normal(scale = 1./self.k, size = (self.num_users, self.k))
        self.Q = np.random.normal(scale = 1./self.k, size = (self.num_items, self.k))
        self.V = np.random.normal(scale = 1./self.k, size = (self.num_items, self.k))
        
        self.b_P = np.zeros(self.num_users)
        self.b_Q = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        self.training_process = []
        time = 0
        start = timer()
        for epoch in range(self.epochs):
            start = timer()
            for u in range(self.num_users):
                for i in range(self.num_items):
                    if self.R[u,i] > 0:
                        self.gradient_descent(u,i,self.R[u,i])
            time += timer() - start
            train_cost, test_cost = self.cost()
            self.training_process.append((epoch, train_cost, test_cost, time))
            
            if self.verbose == True and ((epoch+1) % 10 == 0):
                print('iter: %d, train_cost = %.4f, test_cost = .4%f, average time for 1 epoch: %.4f' % (epoch+1, train_cost, test_cost, time/10))
                time = 0
                
        print("Total time for every epochs: %.4f" % (timer()-start))
        
    def cost(self):
        xi, yi = self.R.nonzero()
        test_x, test_y = self.test.nonzero()
        cost_train = 0
        cost_test = 0
        
        for x,y in zip(xi,yi):
            cost_train += pow(self.R[x,y] - self.get_prediction(x,y), 2)
            
        for i,j in zip(test_x,test_y):
            cost_test += pow(self.test[i,j] - self.get_prediction(i,j), 2)
            
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
    def gradient(self, err, u, i):
        dp = (err * self.Q[i,:]) - (self.reg_param * self.P[u,:])
        dq = (err * (self.P[u,:] + self.N[u] * self.V[self.mask[u,:],:].sum(axis = 0))) - (self.reg_param * self.Q[i,:])
        temp = err * self.N[u] * self.Q[i,:]
        j = self.l[u,:].nonzero()
        dy = temp - self.reg_param * self.V[j,:]
        
        return dp, dq, dy, j
    
    def gradient_descent(self, u, i, rating):
        prediction = self.get_prediction(u,i)
        err = rating - prediction
        
        self.b_P[u] += self.learning_rate * (err - self.reg_param * self.b_P[u])
        self.b_Q[i] += self.learning_rate * (err - self.reg_param * self.b_Q[i])
        
        dp,dq,dy,j = self.gradient(err, u, i)
        self.P[u,:] += self.learning_rate * dp
        self.Q[i,:] += self.learning_rate * dq
        self.V[j,:] += self.learning_rate * dy
        
    def get_prediction(self,u,i):
        x = self.b + self.b_P[u] + self.b_Q[i] + self.Q[i,:].T.dot(self.P[u,:] + self.N[u] * self.V[self.mask[u,:],:].sum(axis = 0))
        return x
    
    def get_complete_matrix(self):
        predictions = np.zeros([self.num_users, self.num_items])
        for u in range(self.num_users):
            for i in range(self.num_items):
                predictions[u,i] = self.get_prediction(u, i)
        return predictions
    
    def print_result(self):
        print('final R predicted: \n')
        print(self.get_complete_matrix())
        print('final RMSE:')
        print(self.training_process[self.epochs - 1][2])
    
    

In [14]:
np.random.seed(7)
np.seterr(all = 'warn')

factorize = MatrixFactorize(train, test, k=20, learning_rate=0.001, reg_param=0.001, epochs=20, verbose = True)

factorize.fit()
factorize.print_result()

iter: 10, train_cost = 0.9386, test_cost = .40.992144, average time for 1 epoch: 17.3208
iter: 20, train_cost = 0.9137, test_cost = .40.970691, average time for 1 epoch: 16.9749
Total time for every epochs: 21.1980
final R predicted: 

[[3.965251   3.29079051 3.06807305 ... 3.52478445 3.49669461 3.63943839]
 [3.93239722 3.44370161 3.30429042 ... 3.58182915 3.60765004 3.63853689]
 [3.48423232 3.09747579 2.95257896 ... 3.21217933 3.19696864 3.26226713]
 ...
 [3.88860278 3.44137713 3.37045175 ... 3.57379085 3.57953416 3.6089843 ]
 [4.28120528 3.78643901 3.63177178 ... 3.93284561 3.96686874 4.02257992]
 [3.77311009 3.23547647 3.0217878  ... 3.40105901 3.3746491  3.48059249]]
final RMSE:
0.9706907895287178
